# このページについて
Mollification Effects of Policy Gradient Methodsという論文について紹介します．これは方策勾配法が，非凸で滑らかな関数でもいい感じに効く手法なのはなぜなのか？という疑問に対して熱方程式という視点からそれをとらえてその問いに答えようとする論文です．

# 問題背景
方策勾配法は深層強化学習を用いたロボット制御をはじめとして，いい感じに動作します．一方で二重振り子での連続値制御を例とするカオス系での学習問題を考えると，最適化したい目的関数が超非滑らかになる場合があります．これをフラクタルなランドスケープとこの論文では言っていますが，幾何学で聞くフラクタルとは違うようで，ただの比喩表現みたいです．フラクタルなランドスケープとは，関数$f$がヘルダー連続であるということとしています．ヘルダー連続の定義を思い出しましょう．

**ヘルダー連続**
関数 $f(x)$ が $\alpha$－ヘルダー連続であるとは、ある定数 $K>0$ が存在し、全ての $x, x^{\prime}$ に対して以下の不等式が成り立つことを指 します。

$$
\left|f(x)-f\left(x^{\prime}\right)\right| \leq K\left|x-x^{\prime}\right|^\alpha
$$

システムのダイナミクスがカオス的(最大リアプノフ指数が正)のとき，目的関数は$\alpha <1$のヘルダー連続になります．つまり，目的関数の真の勾配が存在しない場合があるということです．ヘルダー連続はリプシッツ連続の一般化です．包含関係は，微分可能 ⊂ リプシッツ ⊂ ヘルダー ⊂ 連続です．

|        | リプシッツ連続                                                  | ヘルダー連続                                                                       |
| :----- | :------------------------------------------------------- | :--------------------------------------------------------------------------- |
| 定義式    | $$ \|f(x) - f(y)\| \le K \|x - y\| $$                    | $$ \|f(x) - f(y)\| \le K \|x - y\|^\alpha $$                                 |
| 意味     | 関数の傾きに上界がある                                              | 関数の尖り具合に上界がある                                                                |
| 滑らかさ   | 傾きが無限大になることはないので滑らか．almost everywhereでフレシェ微分可能（ラデマッハの定理） | $\alpha < 1$ の場合、微分不可能な点を持つことができる．傾きが無限大になる点があり得る．                           |
| 条件の強さ  | 強い                                                       | $\alpha < 1$ の場合緩い                                                           |
| 関係性    | ヘルダー連続の $\alpha = 1$ の場合に相当する。                           | リプシッツ連続を一般化した概念。                                                             |
| 例      | $f(x) = \|x\|$                                           | $f(x) = \sqrt{\|x\|}$ ($\alpha=1/2$ のヘルダー連続)                                 |

結局，カオス的なシステムでは勾配が求められないような目的関数を最適化しなければいけません．しかし，しばしば方策勾配法はうまくいく場合があります．それはなぜなのでしょうか？そういったなぞに対して熱方程式という概念を取り入れて答えを出す試みをこの論文は行いました．そのための一つのカギとして，Molificationが挙げられます．
# Molification
これは，関数を滑らかに近似する操作です．具体的にはフラクタルランドスケープな関数に滑らかな関数を畳み込み積分していくことで，元の関数を滑らかな関数にすることを言います．これにより，方策勾配は滑らかな目的関数の上で最適値を探索するためいい感じの働きができるわけです．そしてなんとこの操作が確率的方策(ガウス分布)を使って期待収益を計算する行為がモリフィケーションと一致しているというわけです．

**Molification**
 $f(x)$ に対してガウスカーネルを使ったMolificationは以下のような操作をいう：

$$
f_{\text {mollified }}(x)=\left(f * \phi_\sigma\right)(x)=\int f(y) \phi_\sigma(x-y) d y
$$

なお， $\phi_\sigma(x)=\frac{1}{\sqrt{2 \pi \sigma^2}} e^{-x^2 /\left(2 \sigma^2\right)}$とする．

**期待収益計算**
$$
L_{s}(\theta) := \int_{\mathcal{A}} Q^{\pi}(s, a) \pi_{\theta}(a \mid s) \mathrm{d} a \tag{7}
$$
$Q^\pi$は非平滑になりうる行動価値関数です．重要なのは$\pi$です．これは確率的方策です．これがMolifierの役割を持っています．

ここで、この方策のパラメータ $\theta$ を、$\mu$ と、$\sigma^2$に分けます。$\pi_\theta(a|s) \sim \mathcal{N}(\mu, \sigma^2 I)$なので，これを$L_{s}(\theta) := \int_{\mathcal{A}} Q^{\pi}(s, a) \pi_{\theta}(a \mid s) \mathrm{d} a$に代入すると、期待収益 $L_s$ は $\mu$ と $\sigma^2$ の関数として表現できます．
$$
L_{s}(\mu, \sigma^{2}) = \int_{\mathbb{R}^{m}} Q^{\pi}(s, z) \cdot \frac{1}{(2\pi \sigma^2)^{m/2}} \exp\left(-\frac{\|\mu-z\|^2}{2\sigma^2}\right) \mathrm{d} z
$$
次に、熱方程式を考えます。これは、温度の分布が時間とともにどのように変化するかを記述する方程式です。$\Delta$ラプラシアンで曲率を表します．
$$
\frac{\partial u}{\partial t} = \frac{1}{2} \Delta u 
$$

この方程式は、初期の温度分布 $g(x)$ が与えられたとき、時刻 $t$ における位置 $x$ での温度 $u(x, t)$ が以下で与えられることが知られています。
$$
u(x, t) = \int_{\mathbb{R}^m} g(z) \cdot \frac{1}{(2\pi t)^{m/2}} \exp\left(-\frac{\|x-z\|^2}{2t}\right) \mathrm{d} z
$$
この式の右辺の $g(z)$ 以降の部分はHeat Kernelと呼ばれ、時刻0に点 $z$ に置かれた熱源が、時刻 $t$ に位置 $x$ にどれだけ影響を与えるかを表しています。ここで、先ほどの方策勾配の期待収益の式と、熱方程式の解の公式を並べてみましょう。


| 方策勾配の期待収益                                                                                    | 熱方程式の解                                                            |
| -------------------------------------------------------------------------------------------- | ----------------------------------------------------------------- |
| $L_{s}(\mu, \sigma^{2}) = \int Q^{\pi}(s, z) \cdot \text{Gauss}(\mu, \sigma^2) \mathrm{d} z$ | $u(x, t) = \int g(z) \cdot \text{Heat Kernel}(x, t) \mathrm{d} z$ |

これらはに同じ形をしています．このアナロジー（対応関係）が論文の一番大事なところです．この対応関係を見ていきましょう．

| 方策勾配                 | 熱方程式      | 直感的な意味？              |
| :------------------- | :-------- | :------------------- |
| $\mu$                | $x$       | どこを中心に探索するか          |
| $\sigma^2$           | $t$       | どれだけ多様に探索するか (時間の経過) |
| $L_s(\mu, \sigma^2)$ | $u(x, t)$ | 方策の良さ                |

方策勾配における探索ノイズ，つまり$\sigma^2$は、時間を進めることによる熱拡散です。時間が経つと熱が広がり，温度の分布が滑らかになるように、分散を大きくすると期待収益のランドスケープが滑らかになります。

熱方程式の性質の一つに，強最大値原理があります。これは，熱が伝わっている途中（$t>0$）では、ある一点が周囲のどの点よりも厳密に温度が高くなることはないという原理です。これを方策勾配の文脈で翻訳すると、分散がゼロでない限り（$\sigma^2 > 0$）、期待収益のランドスケープにspurious（偽）なlocal maximumは存在しないということです．つまり、今現在が最適でない限り、必ずより良い方向への勾配が存在し、学習が停滞しないことが保証されます。

# 方策勾配がうまくいかないこともある
一方で，方策勾配がうまくいかない場合があります．熱方程式は時間 $t$が進むにつれて高周波成分を指数関数的に減衰させます．
$$
\tilde{u}(x, t) = \sum_{k=-\infty}^{\infty} B_k \underbrace{e^{-k^2 t}}_{\text{減衰項}} e^{ikx}
$$
減衰項は$e^{-k^2 t}$です．$k$：周波数が大きいほど，減衰は$k^2$のオーダーで速くなります．$t(=\sigma^2)$が大きいほど，全ての周波数で減衰が大きくなります．

このメカニズムから，方策勾配が失敗しやすい問題のクラスが分かります．それは最適解の吸引領域がとても狭く，最適解が高周波なスパイクとして現れる問題です．以下の図を見てください．
![](https://cdn.mathpix.com/cropped/2025_07_13_2aee7fb27cc66fdcc4a6g-09.jpg?height=342&width=771&top_left_y=512&top_left_x=200)
これはドローンを制御タスクです．ドローンのバランスを取るには，左右のモーター推力 $u_1, u_2$ がほぼ等しい必要があります．最適解は $u_1 \approx u_2$ という直線状の領域に存在することになります．図aを参照ください．目的関数のランドスケープは，最適解の周りで非常に鋭いスパイクになります．

方策勾配はガウス分布で行動をサンプリングします．これは，スパイク状のランドスケープを平滑化する畳み込み操作です．分散 $\sigma^2$ がある程度の大きさを持つと，スパイクは削り取られ，平坦になってしまいます． 実験が示しているように，たとえ初期方策を最適解に設定しても，最初の1エポックで平滑化されたランドスケープの勾配を計算した結果，最適解から離れていってしまうわけです．

方策勾配の探索が鋭すぎる最適解を忘れるというデメリットを持つことが言えます．

